# EECS 338 
## Webscraping Notebook

### 04/24/2019
### This is a jupyter notebook to be used to scrape internet data

In [3]:
from __future__ import with_statement
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
browser = webdriver.Chrome('/Users/aea/Downloads/chromedriver')
names = ['Smith', 'Johnson', 'Jones', 'Moen', 'Chen', 'Alwan', 'Briggs', 'Louis',
         "Williams", "Brown", "Garcia", "Rodriguez", "Miller", "Davis"]

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
links = []  # create empty list for all links

for name in names:
    element_names = []    # create temporary list for element names
    
    # perform search by last name
    browser.get('https://www.forevermissed.com/findmemorial/?q=' + name)
    
    # obtain search result line
    search_result = browser.find_elements_by_class_name("bar")
    
    # obtain number of people that match search result
    for res in search_result:
        # convert to string type
        num_results = str(res.text)

        # obtain the number of search results found
        num_results = ''.join(filter(lambda x: x.isdigit(), txt))
        
    # we have the number....
    num_results = int(num_results)    
    offset = 0

    # while loop that resets webpage to display 30 new individuals so that we can obtain 30 more links
    while offset < num_results:
        
        # navigate to webpage with 30 search results
        browser.get('https://www.forevermissed.com/findmemorial/?q=' + name + '&offset=' + str(offset))
        
        # increase the offset by 30 to obtain 30 new results next loop through
        offset += 30
    
        # obtain the link for the individual
        inputElement = browser.find_elements_by_class_name("memorial-link")

        # append the links to a list
        for el in inputElement:
            element_names.append(el.text)

    # append the list of links to a larger list        
    links.append(element_names)
    
links

# remove troublesome links (because they are too long)
links[1].remove('grandpa-fred-uncle-mike-and-my-step-dad-grandma-paula-and-grandma-joanne.forevermissed.com')
links[0].remove('louise-monica-vincent-big-funk-aunt-re-jane-and-sylvester-smith-love-williams-and-pirtle.forevermissed.com')


# output links to a text file
with open('memorial_links.txt', 'w') as f:
    for _list in links:
        f.write(str(_list) + '\n')
            
file.close()




In [ ]:
from bs4 import BeautifulSoup as bs
import requests

tribute_text = []

for i in range(0, len(links)):
    for link in links[i]:
         
        response = requests.get('http://' + link)
        # print(response.text)  

        soup = bs(response.text, "html.parser")
        # print(soup.prettify())

        # obtain the text from written tribute
        user_text = soup.findAll(attrs={'class':'txt user_text'})

        # iterate through each instance and convert text to string type
        for text in user_text:
            tribute_text.append(str(text))

# Do the same for stories tab
stories_text = []
for i in range(0, len(links)):
    for link in links[i]:
         
        response = requests.get('http://' + link + '/#stories')
        # print(response.text)  

        soup = bs(response.text, "html.parser")
        # print(soup.prettify())

        # obtain the text from written tribute
        user_text = soup.findAll(attrs={'class':'user_text'})

        # iterate through each instance and convert text to string type
        for text in user_text:
            stories_text.append(str(text))

In [ ]:
cleaned_tribute_text = []
cleaned_stories_text = []

# clean the data
for item in tribute_text:
    item = item.strip('\n')
    item = item.strip('<div class="txt user_text">')
    item = item.strip('</')
    item = item.replace('\n', ' ')
    cleaned_tribute_text.append(item)
    
for item in stories_text:
    item = item.strip('\n')
    item = item.strip('<div class="txt user_text">')
    item = item.strip('</')
    item = item.replace('\n', ' ')
    cleaned_stories_text.append(item)

# output to a text file
with open('cleaned_tribute_text.txt', 'w') as f:
    for _list in cleaned_tribute_text:
        f.write(str(_list) + '\n')         
file.close()

with open('cleaned_stories_text.txt', 'w') as f:
    for _list in cleaned_stories_text:
        f.write(str(_list) + '\n')         
file.close()

In [ ]:
# # do this for the "Smith" last names
# memories_list = []

# for link in links[3]:
#     browser.get('http://' + link)
    
# #     WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, "page-tributes")))
# #     WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located)
    
# #     WebDriverWait(browser, 2000)
    
# #     inputElement = browser.find_elements_by_css_selector("txt user_text")
#     inputElement = browser.find_elements_by_class_name(".personal_phrase_font.long")
#     inputElement = browser.find_element(By.CSS_SELECTOR, "txt user_text")
#     print(inputElement)
    
    
#     for el in inputElement:
#         print(el.text)
# #         memories_list.append(el.text)

    

